The **Part 1** of the project is divided into three sections:

1- Feature Extraction (Using SIFT)

2- Outlier Removal (Using RANSAC)

3- Computing the Homographies (Using DLT)


**pip install opencv-python**

**pip install opencv-contrib-python**

**Section 1**

In [8]:
#Imports
import numpy as np
import cv2
import os

In [1]:
#Display the video
capture = cv2.VideoCapture(os.path.abspath('trymefirst_lisbon.mp4'))
while(cap.isOpened()):
    ret, frame = cap.read()
    cv2.imshow('frame',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):  ##press q if you want the video to stop 
        break

cap.release()
cv2.destroyAllWindows()

NameError: name 'cv2' is not defined

In [7]:
capture = cv2.VideoCapture(os.path.abspath('trymefirst_lisbon.mp4'))
frameNr = 0
sift = cv2.xfeatures2d.SIFT_create()
while True:
    success, frame = capture.read()
    if success:
        gray= cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
        kp, descriptors = sift.detectAndCompute(frame, None)
        break

print(kp)

AttributeError: module 'cv2' has no attribute 'xfeatures2d'

In [9]:
sift = cv2.xfeatures2d.SIFT_create()

AttributeError: module 'cv2' has no attribute 'xfeatures2d'